In [45]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersORM, ResumesORM, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

#### Без relationship

In [6]:
with session_factory() as session:
    query = (
        select(WorkersORM)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

NameError: name 'session_factory' is not defined

#### C relationship

In [2]:
with session_factory() as session:
    query = (
        select(WorkersORM)
        .options(selectinload(WorkersORM.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

NameError: name 'session_factory' is not defined

#### JOIN

In [3]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

NameError: name 'BaseModel' is not defined

In [53]:
with session_factory() as session:
    query = (
        select(
            ResumesORM.workload,
            func.avg(ResumesORM.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesORM)
        .filter(and_(
            ResumesORM.title.contains("Python"),
            ResumesORM.compensation > 40000,
        ))
        .group_by(ResumesORM.workload)
        .having(func.avg(ResumesORM.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

result_orm=[(<Workload.parttime: 'parttime'>, 165000), (<Workload.fulltime: 'fulltime'>, 90000)]
result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.parttime: 'parttime'>, avg_compensation=165000), WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=90000)]
